In [8]:
##With Apply sharpening filter and Apply deblurring (using Gaussian blur)##
import os
import pandas as pd
import cv2
import numpy as np
from ultralytics import YOLO

# Load YOLO model
model_path = '/work/hdd/bczm/sjafarisheshtamad/COMS571/runs/detect/train59/weights/best.pt'
model = YOLO(model_path)

# Paths to images and ground truth folders
images_folder = '/work/hdd/bczm/sjafarisheshtamad/COMS571/IDCIA v2/images'
ground_truth_folder = '/work/hdd/bczm/sjafarisheshtamad/COMS571/IDCIA v2/ground_truth'

# Fixed confidence rate
confidence_rate = 0.25

# Distance thresholds to evaluate
distance_thresholds = [15, 10]

# Sharpening kernel
sharpening_kernel = np.array([[0, -1, 0],
                               [-1, 5, -1],
                               [0, -1, 0]])

# Initialize results storage for each threshold
results_summary = {threshold: {'total_correct_predictions': 0, 'total_ground_truth_cells': 0} for threshold in distance_thresholds}

# Get the list of all image files in the dataset
image_files = sorted(os.listdir(images_folder))

# Process each image
for image_file in image_files:
    # Skip hidden files or invalid files
    if image_file.startswith('.'):
        continue

    # Load the image
    image_path = os.path.join(images_folder, image_file)
    img = cv2.imread(image_path)
    
    # Apply sharpening filter
    sharpened = cv2.filter2D(img, -1, sharpening_kernel)
    
    # Apply deblurring (using Gaussian blur)
    deblurred = cv2.GaussianBlur(sharpened, (0, 0), 3)
    enhanced_img = cv2.addWeighted(sharpened, 3, deblurred, -1, 0)
    
    # Corresponding CSV file (assuming same base name as the image file)
    base_name = os.path.splitext(image_file)[0]
    csv_file = os.path.join(ground_truth_folder, f"{base_name}.csv")
    
    # Check if the ground truth CSV exists
    if not os.path.exists(csv_file):
        print(f"Ground truth file not found for image: {image_file}")
        continue
    
    # Load ground truth data (X, Y coordinates)
    ground_truth_data = pd.read_csv(csv_file)
    if 'X' not in ground_truth_data.columns or 'Y' not in ground_truth_data.columns:
        print(f"Ground truth CSV for {image_file} does not have 'X' and 'Y' columns.")
        continue
    
    ground_truth_points = ground_truth_data[['X', 'Y']].values
    
    # Count the number of cells in the ground truth
    ground_truth_cells = len(ground_truth_points)
    for threshold in distance_thresholds:
        results_summary[threshold]['total_ground_truth_cells'] += ground_truth_cells
    
    # Run YOLOv8 detection on the enhanced image with the specified confidence
    results = model(enhanced_img, conf=confidence_rate)
    
    # Extract predicted bounding box centroids
    detected_boxes = results[0].boxes
    detected_centroids = [
        [
            ((box.xyxy[0][0] + box.xyxy[0][2]) / 2).cpu().numpy(),
            ((box.xyxy[0][1] + box.xyxy[0][3]) / 2).cpu().numpy()
        ]
        for box in detected_boxes
    ]
    
    # Match predictions to ground truth based on distance thresholds
    for threshold in distance_thresholds:
        correct_predictions = 0
        for gt_point in ground_truth_points:
            for pred_centroid in detected_centroids:
                distance = np.sqrt((gt_point[0] - pred_centroid[0]) ** 2 + (gt_point[1] - pred_centroid[1]) ** 2)
                if distance <= threshold:
                    correct_predictions += 1
                    break  # Match each ground truth point only once
        
        results_summary[threshold]['total_correct_predictions'] += correct_predictions
        
        # Calculate and print accuracy for the current image
        image_accuracy = (correct_predictions / ground_truth_cells) * 100 if ground_truth_cells > 0 else 0
        print(f"Image: {image_file} | Threshold: {threshold}px | Accuracy: {image_accuracy:.2f}%")

# Calculate overall accuracy for each threshold
print("\n--- Final Results ---")
for threshold in distance_thresholds:
    total_correct = results_summary[threshold]['total_correct_predictions']
    total_gt = results_summary[threshold]['total_ground_truth_cells']
    overall_accuracy = (total_correct / total_gt) * 100 if total_gt > 0 else 0
    print(f"Distance Threshold: {threshold}px | Overall Accuracy: {overall_accuracy:.2f}%")



0: 960x1280 252 Cellss, 6.2ms
Speed: 3.7ms preprocess, 6.2ms inference, 1.0ms postprocess per image at shape (1, 3, 960, 1280)
Image: 220812_GFP-AHPC_A_GFAP_F10_DAPI_ND1_20x.tiff | Threshold: 15px | Accuracy: 66.35%
Image: 220812_GFP-AHPC_A_GFAP_F10_DAPI_ND1_20x.tiff | Threshold: 10px | Accuracy: 58.00%

0: 960x1280 300 Cellss, 5.6ms
Speed: 3.8ms preprocess, 5.6ms inference, 1.5ms postprocess per image at shape (1, 3, 960, 1280)
Image: 220812_GFP-AHPC_A_GFAP_F1_DAPI_ND1_20x.tiff | Threshold: 15px | Accuracy: 48.79%
Image: 220812_GFP-AHPC_A_GFAP_F1_DAPI_ND1_20x.tiff | Threshold: 10px | Accuracy: 38.37%

0: 960x1280 146 Cellss, 5.6ms
Speed: 3.8ms preprocess, 5.6ms inference, 0.9ms postprocess per image at shape (1, 3, 960, 1280)
Image: 220812_GFP-AHPC_A_GFAP_F2_DAPI_ND1_20x.tiff | Threshold: 15px | Accuracy: 95.39%
Image: 220812_GFP-AHPC_A_GFAP_F2_DAPI_ND1_20x.tiff | Threshold: 10px | Accuracy: 91.45%

0: 960x1280 300 Cellss, 5.5ms
Speed: 3.8ms preprocess, 5.5ms inference, 1.1ms postpro

In [9]:
##code without preprocessing##

import os
import pandas as pd
import cv2
import numpy as np
from ultralytics import YOLO

# Load YOLO model
model_path = '/work/hdd/bczm/sjafarisheshtamad/COMS571/runs/detect/train59/weights/best.pt'
model = YOLO(model_path)

# Paths to images and ground truth folders
images_folder = '/work/hdd/bczm/sjafarisheshtamad/COMS571/IDCIA v2/images'
ground_truth_folder = '/work/hdd/bczm/sjafarisheshtamad/COMS571/IDCIA v2/ground_truth'

# Fixed confidence rate
confidence_rate = 0.25

# Distance thresholds to evaluate
distance_thresholds = [15, 10]

# Initialize results storage for each threshold
results_summary = {threshold: {'total_correct_predictions': 0, 'total_ground_truth_cells': 0} for threshold in distance_thresholds}

# Get the list of all image files in the dataset
image_files = sorted(os.listdir(images_folder))

# Process each image
for image_file in image_files:
    # Skip hidden files or invalid files
    if image_file.startswith('.'):
        continue

    # Load the image
    image_path = os.path.join(images_folder, image_file)
    img = cv2.imread(image_path)
    
    # No preprocessing applied, use the original image
    enhanced_img = img
    
    # Corresponding CSV file (assuming same base name as the image file)
    base_name = os.path.splitext(image_file)[0]
    csv_file = os.path.join(ground_truth_folder, f"{base_name}.csv")
    
    # Check if the ground truth CSV exists
    if not os.path.exists(csv_file):
        print(f"Ground truth file not found for image: {image_file}")
        continue
    
    # Load ground truth data (X, Y coordinates)
    ground_truth_data = pd.read_csv(csv_file)
    if 'X' not in ground_truth_data.columns or 'Y' not in ground_truth_data.columns:
        print(f"Ground truth CSV for {image_file} does not have 'X' and 'Y' columns.")
        continue
    
    ground_truth_points = ground_truth_data[['X', 'Y']].values
    
    # Count the number of cells in the ground truth
    ground_truth_cells = len(ground_truth_points)
    for threshold in distance_thresholds:
        results_summary[threshold]['total_ground_truth_cells'] += ground_truth_cells
    
    # Run YOLOv8 detection on the original image with the specified confidence
    results = model(enhanced_img, conf=confidence_rate)
    
    # Extract predicted bounding box centroids
    detected_boxes = results[0].boxes
    detected_centroids = [
        [
            ((box.xyxy[0][0] + box.xyxy[0][2]) / 2).cpu().numpy(),
            ((box.xyxy[0][1] + box.xyxy[0][3]) / 2).cpu().numpy()
        ]
        for box in detected_boxes
    ]
    
    # Match predictions to ground truth based on distance thresholds
    for threshold in distance_thresholds:
        correct_predictions = 0
        for gt_point in ground_truth_points:
            for pred_centroid in detected_centroids:
                distance = np.sqrt((gt_point[0] - pred_centroid[0]) ** 2 + (gt_point[1] - pred_centroid[1]) ** 2)
                if distance <= threshold:
                    correct_predictions += 1
                    break  # Match each ground truth point only once
        
        results_summary[threshold]['total_correct_predictions'] += correct_predictions
        
        # Calculate and print accuracy for the current image
        image_accuracy = (correct_predictions / ground_truth_cells) * 100 if ground_truth_cells > 0 else 0
        print(f"Image: {image_file} | Threshold: {threshold}px | Accuracy: {image_accuracy:.2f}%")

# Calculate overall accuracy for each threshold
print("\n--- Final Results ---")
for threshold in distance_thresholds:
    total_correct = results_summary[threshold]['total_correct_predictions']
    total_gt = results_summary[threshold]['total_ground_truth_cells']
    overall_accuracy = (total_correct / total_gt) * 100 if total_gt > 0 else 0
    print(f"Distance Threshold: {threshold}px | Overall Accuracy: {overall_accuracy:.2f}%")



0: 960x1280 91 Cellss, 5.5ms
Speed: 4.3ms preprocess, 5.5ms inference, 0.9ms postprocess per image at shape (1, 3, 960, 1280)
Image: 220812_GFP-AHPC_A_GFAP_F10_DAPI_ND1_20x.tiff | Threshold: 15px | Accuracy: 27.68%
Image: 220812_GFP-AHPC_A_GFAP_F10_DAPI_ND1_20x.tiff | Threshold: 10px | Accuracy: 21.96%

0: 960x1280 106 Cellss, 5.6ms
Speed: 3.6ms preprocess, 5.6ms inference, 0.9ms postprocess per image at shape (1, 3, 960, 1280)
Image: 220812_GFP-AHPC_A_GFAP_F1_DAPI_ND1_20x.tiff | Threshold: 15px | Accuracy: 21.98%
Image: 220812_GFP-AHPC_A_GFAP_F1_DAPI_ND1_20x.tiff | Threshold: 10px | Accuracy: 14.10%

0: 960x1280 49 Cellss, 5.6ms
Speed: 4.1ms preprocess, 5.6ms inference, 0.9ms postprocess per image at shape (1, 3, 960, 1280)
Image: 220812_GFP-AHPC_A_GFAP_F2_DAPI_ND1_20x.tiff | Threshold: 15px | Accuracy: 40.13%
Image: 220812_GFP-AHPC_A_GFAP_F2_DAPI_ND1_20x.tiff | Threshold: 10px | Accuracy: 30.92%

0: 960x1280 76 Cellss, 5.1ms
Speed: 3.4ms preprocess, 5.1ms inference, 0.8ms postproces

In [11]:
##code with only debluring##

import os
import pandas as pd
import cv2
import numpy as np
from ultralytics import YOLO

# Load YOLO model
model_path = '/work/hdd/bczm/sjafarisheshtamad/COMS571/runs/detect/train59/weights/best.pt'
model = YOLO(model_path)

# Paths to images and ground truth folders
images_folder = '/work/hdd/bczm/sjafarisheshtamad/COMS571/IDCIA v2/images'
ground_truth_folder = '/work/hdd/bczm/sjafarisheshtamad/COMS571/IDCIA v2/ground_truth'

# Fixed confidence rate
confidence_rate = 0.25

# Distance thresholds to evaluate
distance_thresholds = [15, 10]

# Initialize results storage for each threshold
results_summary = {threshold: {'total_correct_predictions': 0, 'total_ground_truth_cells': 0} for threshold in distance_thresholds}

# Get the list of all image files in the dataset
image_files = sorted(os.listdir(images_folder))

# Process each image
for image_file in image_files:
    # Skip hidden files or invalid files
    if image_file.startswith('.'):
        continue

    # Load the image
    image_path = os.path.join(images_folder, image_file)
    img = cv2.imread(image_path)
    
    # Apply deblurring (using Gaussian blur)
    deblurred = cv2.GaussianBlur(img, (0, 0), 3)
    enhanced_img = cv2.addWeighted(img, 3, deblurred, -1, 0)
    
    # Corresponding CSV file (assuming same base name as the image file)
    base_name = os.path.splitext(image_file)[0]
    csv_file = os.path.join(ground_truth_folder, f"{base_name}.csv")
    
    # Check if the ground truth CSV exists
    if not os.path.exists(csv_file):
        print(f"Ground truth file not found for image: {image_file}")
        continue
    
    # Load ground truth data (X, Y coordinates)
    ground_truth_data = pd.read_csv(csv_file)
    if 'X' not in ground_truth_data.columns or 'Y' not in ground_truth_data.columns:
        print(f"Ground truth CSV for {image_file} does not have 'X' and 'Y' columns.")
        continue
    
    ground_truth_points = ground_truth_data[['X', 'Y']].values
    
    # Count the number of cells in the ground truth
    ground_truth_cells = len(ground_truth_points)
    for threshold in distance_thresholds:
        results_summary[threshold]['total_ground_truth_cells'] += ground_truth_cells
    
    # Run YOLOv8 detection on the enhanced image with the specified confidence
    results = model(enhanced_img, conf=confidence_rate)
    
    # Extract predicted bounding box centroids
    detected_boxes = results[0].boxes
    detected_centroids = [
        [
            ((box.xyxy[0][0] + box.xyxy[0][2]) / 2).cpu().numpy(),
            ((box.xyxy[0][1] + box.xyxy[0][3]) / 2).cpu().numpy()
        ]
        for box in detected_boxes
    ]
    
    # Match predictions to ground truth based on distance thresholds
    for threshold in distance_thresholds:
        correct_predictions = 0
        for gt_point in ground_truth_points:
            for pred_centroid in detected_centroids:
                distance = np.sqrt((gt_point[0] - pred_centroid[0]) ** 2 + (gt_point[1] - pred_centroid[1]) ** 2)
                if distance <= threshold:
                    correct_predictions += 1
                    break  # Match each ground truth point only once
        
        results_summary[threshold]['total_correct_predictions'] += correct_predictions
        
        # Calculate and print accuracy for the current image
        image_accuracy = (correct_predictions / ground_truth_cells) * 100 if ground_truth_cells > 0 else 0
        print(f"Image: {image_file} | Threshold: {threshold}px | Accuracy: {image_accuracy:.2f}%")

# Calculate overall accuracy for each threshold
print("\n--- Final Results ---")
for threshold in distance_thresholds:
    total_correct = results_summary[threshold]['total_correct_predictions']
    total_gt = results_summary[threshold]['total_ground_truth_cells']
    overall_accuracy = (total_correct / total_gt) * 100 if total_gt > 0 else 0
    print(f"Distance Threshold: {threshold}px | Overall Accuracy: {overall_accuracy:.2f}%")



0: 960x1280 300 Cellss, 5.7ms
Speed: 5.2ms preprocess, 5.7ms inference, 1.0ms postprocess per image at shape (1, 3, 960, 1280)
Image: 220812_GFP-AHPC_A_GFAP_F10_DAPI_ND1_20x.tiff | Threshold: 15px | Accuracy: 78.28%
Image: 220812_GFP-AHPC_A_GFAP_F10_DAPI_ND1_20x.tiff | Threshold: 10px | Accuracy: 68.02%

0: 960x1280 300 Cellss, 5.6ms
Speed: 3.8ms preprocess, 5.6ms inference, 1.2ms postprocess per image at shape (1, 3, 960, 1280)
Image: 220812_GFP-AHPC_A_GFAP_F1_DAPI_ND1_20x.tiff | Threshold: 15px | Accuracy: 49.68%
Image: 220812_GFP-AHPC_A_GFAP_F1_DAPI_ND1_20x.tiff | Threshold: 10px | Accuracy: 39.64%

0: 960x1280 122 Cellss, 5.6ms
Speed: 3.7ms preprocess, 5.6ms inference, 1.0ms postprocess per image at shape (1, 3, 960, 1280)
Image: 220812_GFP-AHPC_A_GFAP_F2_DAPI_ND1_20x.tiff | Threshold: 15px | Accuracy: 86.84%
Image: 220812_GFP-AHPC_A_GFAP_F2_DAPI_ND1_20x.tiff | Threshold: 10px | Accuracy: 76.32%

0: 960x1280 300 Cellss, 5.4ms
Speed: 3.7ms preprocess, 5.4ms inference, 1.0ms postpro

In [10]:
##Code with Only Sharpening##
import os
import pandas as pd
import cv2
import numpy as np
from ultralytics import YOLO

# Load YOLO model
model_path = '/work/hdd/bczm/sjafarisheshtamad/COMS571/runs/detect/train59/weights/best.pt'
model = YOLO(model_path)

# Paths to images and ground truth folders
images_folder = '/work/hdd/bczm/sjafarisheshtamad/COMS571/IDCIA v2/images'
ground_truth_folder = '/work/hdd/bczm/sjafarisheshtamad/COMS571/IDCIA v2/ground_truth'

# Fixed confidence rate
confidence_rate = 0.25

# Distance thresholds to evaluate
distance_thresholds = [15, 10]

# Sharpening kernel
sharpening_kernel = np.array([[0, -1, 0],
                               [-1, 5, -1],
                               [0, -1, 0]])

# Initialize results storage for each threshold
results_summary = {threshold: {'total_correct_predictions': 0, 'total_ground_truth_cells': 0} for threshold in distance_thresholds}

# Get the list of all image files in the dataset
image_files = sorted(os.listdir(images_folder))

# Process each image
for image_file in image_files:
    # Skip hidden files or invalid files
    if image_file.startswith('.'):
        continue

    # Load the image
    image_path = os.path.join(images_folder, image_file)
    img = cv2.imread(image_path)
    
    # Apply sharpening filter
    enhanced_img = cv2.filter2D(img, -1, sharpening_kernel)
    
    # Corresponding CSV file (assuming same base name as the image file)
    base_name = os.path.splitext(image_file)[0]
    csv_file = os.path.join(ground_truth_folder, f"{base_name}.csv")
    
    # Check if the ground truth CSV exists
    if not os.path.exists(csv_file):
        print(f"Ground truth file not found for image: {image_file}")
        continue
    
    # Load ground truth data (X, Y coordinates)
    ground_truth_data = pd.read_csv(csv_file)
    if 'X' not in ground_truth_data.columns or 'Y' not in ground_truth_data.columns:
        print(f"Ground truth CSV for {image_file} does not have 'X' and 'Y' columns.")
        continue
    
    ground_truth_points = ground_truth_data[['X', 'Y']].values
    
    # Count the number of cells in the ground truth
    ground_truth_cells = len(ground_truth_points)
    for threshold in distance_thresholds:
        results_summary[threshold]['total_ground_truth_cells'] += ground_truth_cells
    
    # Run YOLOv8 detection on the sharpened image with the specified confidence
    results = model(enhanced_img, conf=confidence_rate)
    
    # Extract predicted bounding box centroids
    detected_boxes = results[0].boxes
    detected_centroids = [
        [
            ((box.xyxy[0][0] + box.xyxy[0][2]) / 2).cpu().numpy(),
            ((box.xyxy[0][1] + box.xyxy[0][3]) / 2).cpu().numpy()
        ]
        for box in detected_boxes
    ]
    
    # Match predictions to ground truth based on distance thresholds
    for threshold in distance_thresholds:
        correct_predictions = 0
        for gt_point in ground_truth_points:
            for pred_centroid in detected_centroids:
                distance = np.sqrt((gt_point[0] - pred_centroid[0]) ** 2 + (gt_point[1] - pred_centroid[1]) ** 2)
                if distance <= threshold:
                    correct_predictions += 1
                    break  # Match each ground truth point only once
        
        results_summary[threshold]['total_correct_predictions'] += correct_predictions
        
        # Calculate and print accuracy for the current image
        image_accuracy = (correct_predictions / ground_truth_cells) * 100 if ground_truth_cells > 0 else 0
        print(f"Image: {image_file} | Threshold: {threshold}px | Accuracy: {image_accuracy:.2f}%")

# Calculate overall accuracy for each threshold
print("\n--- Final Results ---")
for threshold in distance_thresholds:
    total_correct = results_summary[threshold]['total_correct_predictions']
    total_gt = results_summary[threshold]['total_ground_truth_cells']
    overall_accuracy = (total_correct / total_gt) * 100 if total_gt > 0 else 0
    print(f"Distance Threshold: {threshold}px | Overall Accuracy: {overall_accuracy:.2f}%")




0: 960x1280 223 Cellss, 5.5ms
Speed: 3.6ms preprocess, 5.5ms inference, 1.0ms postprocess per image at shape (1, 3, 960, 1280)
Image: 220812_GFP-AHPC_A_GFAP_F10_DAPI_ND1_20x.tiff | Threshold: 15px | Accuracy: 60.62%
Image: 220812_GFP-AHPC_A_GFAP_F10_DAPI_ND1_20x.tiff | Threshold: 10px | Accuracy: 51.55%

0: 960x1280 300 Cellss, 5.5ms
Speed: 4.3ms preprocess, 5.5ms inference, 1.3ms postprocess per image at shape (1, 3, 960, 1280)
Image: 220812_GFP-AHPC_A_GFAP_F1_DAPI_ND1_20x.tiff | Threshold: 15px | Accuracy: 49.17%
Image: 220812_GFP-AHPC_A_GFAP_F1_DAPI_ND1_20x.tiff | Threshold: 10px | Accuracy: 38.63%

0: 960x1280 127 Cellss, 5.8ms
Speed: 3.7ms preprocess, 5.8ms inference, 1.0ms postprocess per image at shape (1, 3, 960, 1280)
Image: 220812_GFP-AHPC_A_GFAP_F2_DAPI_ND1_20x.tiff | Threshold: 15px | Accuracy: 88.16%
Image: 220812_GFP-AHPC_A_GFAP_F2_DAPI_ND1_20x.tiff | Threshold: 10px | Accuracy: 80.26%

0: 960x1280 300 Cellss, 5.5ms
Speed: 3.5ms preprocess, 5.5ms inference, 1.0ms postpro